# Implementation to consider sample intervals

## Imports

In [1]:
from striplog import Component, Decor, Legend, Lexicon, Striplog
from core.visual import Borehole3D, striplog_legend_to_omf_legend 
from utils.lexicon.lexicon_memoris import LEG_CONTAMINATION_LEV
from utils.config import DEFAULT_LITHO_LEXICON, DEFAULT_LITHO_LEGEND, DEFAULT_POL_LEXICON
from utils.io import dataframe_viewer, dict_viewer
from utils.orm import boreholes_from_dataframe
from utils.visual import legend_from_attributes
from core.project import Project
from core.orm import BoreholeOrm, PositionOrm, Base 

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from definitions import ROOT_DIR


TEST_ORM---------: (no name)



## Reading of CSV files in a dir

In [2]:
df = pd.read_csv('real_data_subset_for_test.csv', sep=',')

In [3]:
dataframe_viewer(df, rows=5, un_val='ID')

Rows : 112, columns : 19, Unique values on col 'ID': 46


interactive(children=(IntSlider(value=5, description='rows', max=112, min=5, readout=False), IntSlider(value=1…

In [4]:
df_litho = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'Litho_top', 'Litho_base', 'Epais_intv', 'Lithology']]
df_litho.rename(columns={'Lithology':'Description'}, inplace=True)

df_ech = df[['ID', 'X', 'Y', 'Z', 'Long_for', 'ID_ech', 'Type_ech', 'Top_intv', 'Base_intv', 
             'Epais_intv', 'Bnz', 'Toln', 'Pb', 'As', 'Cu', 'Ni']]
df_ech.rename(columns={'Top_intv':'Ech_top', 'Base_intv':'Ech_base'}, inplace=True)

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [5]:
pollutants = []
for i, c in enumerate(df.columns):
    if c in DEFAULT_POL_LEXICON.abbreviations.keys() or c in DEFAULT_POL_LEXICON.abbreviations.values():
        print(f"{i}: {c}")
        pollutants.append(c)

13: Bnz
14: Toln
15: Pb
16: As
17: Cu
18: Ni


In [6]:
bh_id='201'
dataframe_viewer(df_litho.query(f"ID=='{bh_id}'"), rows=3, cols=16), dataframe_viewer(df_ech.query(f"ID=='{bh_id}'"), rows=3, cols=16)

Rows : 2, columns : 9


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=9, …

Rows : 2, columns : 16


interactive(children=(IntSlider(value=2, description='rows', max=2, min=2, readout=False), IntSlider(value=16,…

(None, None)

In [7]:
# adds a description in sample (TEST)
df_ech['Description'] = np.nan
df_ech.loc[:1, 'Description'] = ['Sable', 'Limon']

/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)
/home/yanathan/.local/share/virtualenvs/GSDMA-DRfwm83x/lib/python3.8/site-packages/pandas/core/indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


## Create Boreholes from dataframes

In [8]:
data_dict = {'lithologies_data': df_litho, 'pollutants_data': df_ech}

In [9]:
boreholes, components, link_intv_comp = boreholes_from_dataframe(data_dict, verbose=False,
                                                sample_type_col='Type_ech', diameter_col='Diam_for', 
                                                average_z=102, attributes=['lithology']+pollutants, 
                                                symbols={'lithology':{'lexicon':DEFAULT_LITHO_LEXICON}})


Data Processing...
To continue, default diameter column has been created with value: 0.1 [m]

 BH_ID: '201'
0- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0.6, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

2- Interval top=0.7, base=1.2, type=sample
 - Interval components: [Component({'lithology': 'sable'}), Component({'benzène': 'vi'}), Component({'toluène': 'vs'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]

3- Interval top=2.0, base=2.4, type=sam

/home/yanathan/Projects/GSDMA/utils/orm.py:207: SAWarning: relationship 'ComponentOrm.intervals' will copy column Components.id to column Linkintervalcomponent.comp_id, which conflicts with relationship(s): 'IntervalOrm.components' (copies Components.id to Linkintervalcomponent.comp_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="components"' to the 'ComponentOrm.intervals' relationship.
  boreholes_orm.append(BoreholeOrm(id=bh_name, date=bh_date))
/home/yanathan/Projects/GSDMA/utils/orm.py:207: SAWarning: relationship 'ComponentOrm.intervals' will copy column Intervals.id to column Linkintervalcomponent.intv_id, which conflic


 BH_ID: '205'
0- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

2- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

3- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'boue'}), Component({'benzène': 'vi'}), Comp

3- Interval top=0.0, base=1.2, type=lithology
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

4- Interval top=0.0, base=0.5, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vs'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vs'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]

5- Interval top=0.6, base=1.0, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vs'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]

6- Interval top=3.4, base=3.6, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vr'}), Component({'cuivre': 'vr'}), Component


 BH_ID: '304'
0- Interval top=0.0, base=0.6, type=lithology
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0.85, base=1.15, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vs'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vs'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]

 Summary : {'304': {'lithology': Striplog(1 Intervals, start=0.0, stop=0.6), 'sample': Striplog(1 Intervals, start=0.85, stop=1.15)}}

 BH_ID: '305'
0- Interval top=0.0, base=0.8, type=lithology
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0

 Summary : {'F2M': {'lithology': Striplog(3 Intervals, start=0.0, stop=4.0), 'sample': Striplog(3 Intervals, start=0.3, stop=6.0)}}

 BH_ID: 'F3M'
0- Interval top=0.0, base=0.2, type=lithology
 - Interval components: [Component({'lithology': 'béton'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0.0, base=0.5, type=lithology
 - Interval components: [Component({'lithology': 'béton'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

2- Interval top=2.4, base=3.3, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vr'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vs'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]

3- Interval top=0.0, base=0.5, type=sample

 BH_ID: 'F27a'
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

1- Interval top=0.0, base=0.5, type=sample
 - Interval components: [Component({'benzène': 'vi'}), Component({'toluène': 'vs'}), Component({'plomb': 'vs'}), Component({'arsenic': 'vs'}), Component({'cuivre': 'vs'}), Component({'nickel': 'vs'})]


 Summary : {'F27a': {'sample': Striplog(1 Intervals, start=0.0, stop=0.5)}}

 BH_ID: 'F27d'
 - Interval components: [Component({'lithology': 'remblais'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arsenic': 'vi'}), Component({'cuivre': 'vi'}), Component({'nickel': 'vi'})]

 - Interval components: [Component({'lithology': 'limons'}), Component({'benzène': 'vi'}), Component({'toluène': 'vi'}), Component({'plomb': 'vi'}), Component({'arseni

In [10]:
len(boreholes), len(components)

(46, 25)

In [11]:
boreholes[0].litho_intv_values

{2: "{'lithology': 'sable', 'benzène': 'vi', 'toluène': 'vs', 'plomb': 'vs', 'arsenic': 'vr', 'cuivre': 'vs', 'nickel': 'vs'}", 3: "{'lithology': 'limon', 'benzène': 'vs', 'toluène': 'vr', 'plomb': 'vi', 'arsenic': 'vs', 'cuivre': 'vr', 'nickel': 'vs'}"}

In [12]:
boreholes[0].sample_intv_values

{2: "{'lithology': 'sable', 'benzène': 'vi', 'toluène': 'vs', 'plomb': 'vs', 'arsenic': 'vr', 'cuivre': 'vs', 'nickel': 'vs'}", 3: "{'lithology': 'limon', 'benzène': 'vs', 'toluène': 'vr', 'plomb': 'vi', 'arsenic': 'vs', 'cuivre': 'vr', 'nickel': 'vs'}"}

In [13]:
test = {'litho':{0:{'value':'sable'}}}
test
s = lambda k, v: test[k][v]
s('litho', 0)

{'value': 'sable'}

In [14]:
boreholes

[<core.orm.BoreholeOrm>(Name=201, Length=1.7999999999999998, Diameter=0.1, Intervals=2),
 <core.orm.BoreholeOrm>(Name=205, Length=3.0, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=208, Length=2.999999999999999, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=207, Length=3.600000000000001, Diameter=0.1, Intervals=5),
 <core.orm.BoreholeOrm>(Name=212, Length=2.9999999999999982, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=214, Length=3.399999999999999, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=217, Length=3.000000000000001, Diameter=0.1, Intervals=4),
 <core.orm.BoreholeOrm>(Name=221, Length=0.8, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=223, Length=1.6, Diameter=0.1, Intervals=2),
 <core.orm.BoreholeOrm>(Name=?2, Length=0.7999999999999998, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=?1, Length=0.7999999999999998, Diameter=0.1, Intervals=1),
 <core.orm.BoreholeOrm>(Name=224, Length=1.0, Diameter=0.1, Intervals=1)

In [15]:
components

{0: Component({'lithology': 'remblai'}),
 1: Component({'benzène': 'vi'}),
 2: Component({'toluène': 'vi'}),
 3: Component({'plomb': 'vi'}),
 4: Component({'arsenic': 'vi'}),
 5: Component({'cuivre': 'vi'}),
 6: Component({'nickel': 'vi'}),
 7: Component({'lithology': 'sable'}),
 8: Component({'toluène': 'vs'}),
 9: Component({'plomb': 'vs'}),
 10: Component({'arsenic': 'vr'}),
 11: Component({'cuivre': 'vs'}),
 12: Component({'nickel': 'vs'}),
 13: Component({'lithology': 'limon'}),
 14: Component({'benzène': 'vs'}),
 15: Component({'toluène': 'vr'}),
 16: Component({'arsenic': 'vs'}),
 17: Component({'cuivre': 'vr'}),
 18: Component({'lithology': 'boue'}),
 19: Component({'nickel': 'vr'}),
 20: Component({'plomb': 'vr'}),
 21: Component({'benzène': 'vr'}),
 22: Component({'lithology': 'schiste'}),
 23: Component({'lithology': 'béton'}),
 24: Component({'lithology': 'alluvion'})}

In [16]:
i = 0
components[i][list(components[i].keys())[0]]#.json()

'remblai'

## Create a dictionary containing legends for each representation attribute

In [23]:
legend_dict = legend_from_attributes([('lithology', DEFAULT_LITHO_LEGEND)]+pollutants)

In [24]:
dict_viewer(legend_dict)

lithology,"{'legend': Legend(Decor({'_colour': '#ffffe9', 'width': None, 'hatch': None, 'component': Component({'lithology': 'matériau(?:x)? meuble(?:s)?'})}) Decor({'_colour': '#fff497', 'width': None, 'hatch': ""'....'"", 'component': Component({'lithology': 'alluvion'})}) Decor({'_colour': '#b54500', 'width': None, 'hatch': None, 'component': Component({'lithology': 'boue'})}) Decor({'_colour': '#d3b798', 'width': None, 'hatch': ""'v'"", 'component': Component({'lithology': 'remblai'})}) Decor({'_colour': '#a5c7c9', 'width': None, 'hatch': ""'t'"", 'component': Component({'lithology': 'b[é|e]ton'})}) Decor({'_colour': '#8da3c9', 'width': None, 'hatch': ""'t'"", 'component': Component({'lithology': 'scorie'})}) Decor({'_colour': '#ffcc99', 'width': None, 'hatch': None, 'component': Component({'lithology': 'tourbe'})}) Decor({'_colour': '#ffeaa7', 'width': None, 'hatch': None, 'component': Component({'lithology': 'gypse'})}) Decor({'_colour': '#00151a', 'width': None, 'hatch': None, 'component': Component({'lithology': 'houille'})}) Decor({'_colour': '#798732', 'width': None, 'hatch': ""'-.'"", 'component': Component({'lithology': 'limon'})}) Decor({'_colour': '#ffcb23', 'width': None, 'hatch': ""'..'"", 'component': Component({'lithology': 'sable'})}) Decor({'_colour': '#adb7cc', 'width': None, 'hatch': ""'oo"", 'component': Component({'lithology': 'gravier'})}) Decor({'_colour': '#ace4c8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'silt'})}) Decor({'_colour': '#d5e6cc', 'width': None, 'hatch': ""'---'"", 'component': Component({'lithology': 'argile'})}) Decor({'_colour': '#92dcb7', 'width': None, 'hatch': None, 'component': Component({'lithology': 'bentonite'})}) Decor({'_colour': '#bbffdd', 'width': None, 'hatch': ""'--'"", 'component': Component({'lithology': 'schiste'})}) Decor({'_colour': '#95ffca', 'width': None, 'hatch': None, 'component': Component({'lithology': 'argilite'})}) Decor({'_colour': '#d6fe9a', 'width': None, 'hatch': None, 'component': Component({'lithology': 'siltite'})}) Decor({'_colour': '#e1f0d8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'tuffeau'})}) Decor({'_colour': '#69cf9c', 'width': None, 'hatch': None, 'component': Component({'lithology': 'silex'})}) Decor({'_colour': '#b7d9cc', 'width': None, 'hatch': None, 'component': Component({'lithology': 'conglomérat'})}) Decor({'_colour': '#019ccd', 'width': None, 'hatch': None, 'component': Component({'lithology': 'carbonate'})}) Decor({'_colour': '#149eff', 'width': None, 'hatch': ""'='"", 'component': Component({'lithology': 'calcaire'})}) Decor({'_colour': '#fdaaff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'dolomite'})}) Decor({'_colour': '#fdaffe', 'width': None, 'hatch': None, 'component': Component({'lithology': 'dolomie'})}) Decor({'_colour': '#deeffe', 'width': None, 'hatch': None, 'component': Component({'lithology': 'craie'})}) Decor({'_colour': '#aac2c8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'chert'})}) Decor({'_colour': '#000000', 'width': None, 'hatch': None, 'component': Component({'lithology': 'charbon'})}) Decor({'_colour': '#7ba1a8', 'width': None, 'hatch': None, 'component': Component({'lithology': 'marne'})}) Decor({'_colour': '#ffffff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'inconnu'})}) Decor({'_colour': '#ffffff', 'width': None, 'hatch': None, 'component': Component({'lithology': 'not_exist'})}))}"
benzène,"{'legend': Legend(Decor({'_colour': '#00ff00', 'width': None, 'component': Component({'benzène': 'vr'}), 'hatch': None}) Decor({'_colour': '#ffa500', 'width': None, 'component': Component({'benzène': 'vs'}), 'hatch': None}) Decor({'_colour': '#ff0000', 'width': None, 'component': Component({'benzène': 'vi'}), 'hatch': None}) Decor({'_colour': '#ffffff', 'width': None, 'component': Component({'benzène': 'inconnu'}), 'hatch': None}))}"
toluène,"{'legend': Legend(Decor({'_colour': '#00ff00', 'wid

## Create a project and save data in a database

In [25]:
DB_name = 'tmp_files/real_orm_db.db'

if os.path.exists(DB_name):
    os.remove(DB_name)

engine = create_engine(f"sqlite:///{DB_name}", echo=True)

In [26]:
Base.metadata.create_all(engine)

2021-11-01 23:21:25,394 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:25,395 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Boreholes")
2021-11-01 23:21:25,396 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-01 23:21:25,397 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Boreholes")
2021-11-01 23:21:25,398 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-01 23:21:25,399 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Intervals")
2021-11-01 23:21:25,400 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-01 23:21:25,400 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Intervals")
2021-11-01 23:21:25,401 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-01 23:21:25,402 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("Components")
2021-11-01 23:21:25,402 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-11-01 23:21:25,403 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("Components")
2021-11-01 23:21:25,403 INFO sqlalchemy.engine.Engin

In [27]:
Session = sessionmaker(bind=engine)
session = Session()

#### Set project's legend dictionary

In [28]:
p = Project(session, name='Memoris_project', legend_dict=legend_dict, lexicon=DEFAULT_LITHO_LEXICON)

2021-11-01 23:21:25,473 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:25,477 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-11-01 23:21:25,478 INFO sqlalchemy.engine.Engine [generated in 0.00126s] ()


#### Add components, boreholes into the project

In [29]:
p.add_components(components)

2021-11-01 23:21:25,487 INFO sqlalchemy.engine.Engine INSERT INTO "Components" (id, description) VALUES (?, ?)
2021-11-01 23:21:25,488 INFO sqlalchemy.engine.Engine [generated in 0.00138s] ((0, "{'lithology': 'remblai'}"), (1, "{'benzène': 'vi'}"), (2, "{'toluène': 'vi'}"), (3, "{'plomb': 'vi'}"), (4, "{'arsenic': 'vi'}"), (5, "{'cuivre': 'vi'}"), (6, "{'nickel': 'vi'}"), (7, "{'lithology': 'sable'}")  ... displaying 10 of 25 total bound parameter sets ...  (23, "{'lithology': 'béton'}"), (24, "{'lithology': 'alluvion'}"))
2021-11-01 23:21:25,491 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  0
2021-11-01 23:21:25,495 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:25,496 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-11-01 23:21:25,497 INFO sqlalchemy.engine.Engine [cached since 0.0

In [30]:
for bh in boreholes:
    p.add_borehole(bh)

2021-11-01 23:21:25,504 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-11-01 23:21:25,505 INFO sqlalchemy.engine.Engine [generated in 0.00128s] ('201', None, 1.7999999999999998, 0.1)
2021-11-01 23:21:25,508 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-11-01 23:21:25,508 INFO sqlalchemy.engine.Engine [generated in 0.00089s] ((4, 101.226886, 101.226886, 101.226886, 152890.245758, 122596.47426), (5, 100.726886, 100.726886, 100.726886, 152890.245758, 122596.47426), (6, 99.926886, 99.926886, 99.926886, 152890.245758, 122596.47426), (7, 99.52688599999999, 99.52688599999999, 99.52688599999999, 152890.245758, 122596.47426))
2021-11-01 23:21:25,510 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-11-01 23:21:25,511 INFO sqlalchemy.engine.Engine [

2021-11-01 23:21:25,582 INFO sqlalchemy.engine.Engine [cached since 0.05342s ago] (18,)
2021-11-01 23:21:25,584 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:25,585 INFO sqlalchemy.engine.Engine [cached since 0.0525s ago] (19,)
2021-11-01 23:21:25,586 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:25,587 INFO sqlalchemy.engine.Engine [cached since 0.05818s ago] (20,)
2021-11-01 23:21:25,588 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions

2021-11-01 23:21:25,646 INFO sqlalchemy.engine.Engine [cached since 0.1141s ago] (39,)
No lithology interval found, a default one is created for visualization !

Borehole geometry created successfully !

2021-11-01 23:21:25,666 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-11-01 23:21:25,667 INFO sqlalchemy.engine.Engine [cached since 0.1632s ago] ('207', None, 3.600000000000001, 0.1)
2021-11-01 23:21:25,669 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-11-01 23:21:25,669 INFO sqlalchemy.engine.Engine [cached since 0.1616s ago] ((50, 101.78913200000001, 101.78913200000001, 101.78913200000001, 152892.92499700002, 122592.662277), (51, 101.389132, 101.389132, 101.389132, 152892.92499700002, 122592.662277), (52, 101.28913200000001, 101.28913200000001, 101.28913200000001, 152892.92499700002, 122592.662277), (53, 100.889132, 100.889132, 100.889132, 152892.9249

2021-11-01 23:21:25,732 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-11-01 23:21:25,733 INFO sqlalchemy.engine.Engine [cached since 0.2225s ago] ((34, '212', 4, 'sample', "{'benzène': 'vi', 'toluène': 'vr', 'plomb': 'vs', 'arsenic': 'vi', 'cuivre': 'vs', 'nickel': 'vs'}", 68, 69, None), (35, '212', 5, 'sample', "{'benzène': 'vi', 'toluène': 'vs', 'plomb': 'vs', 'arsenic': 'vs', 'cuivre': 'vs', 'nickel': 'vs'}", 70, 71, None), (36, '212', 6, 'sample', "{'benzène': 'vi', 'toluène': 'vr', 'plomb': 'vs', 'arsenic': 'vr', 'cuivre': 'vr', 'nickel': 'vr'}", 72, 73, None), (37, '212', 7, 'sample', "{'benzène': 'vr', 'toluène': 'vr', 'plomb': 'vr', 'arsenic': 'vr', 'cuivre': 'vr', 'nickel': 'vs'}", 74, 75, None))
2021-11-01 23:21:25,734 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  4
2021-11-01 23:21:25,738 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2

2021-11-01 23:21:25,805 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:25,805 INFO sqlalchemy.engine.Engine [cached since 0.2767s ago] (84,)
2021-11-01 23:21:25,807 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:25,808 INFO sqlalchemy.engine.Engine [cached since 0.2754s ago] (85,)
2021-11-01 23:21:25,810 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "

...

2021-11-01 23:21:27,509 INFO sqlalchemy.engine.Engine [cached since 2.005s ago] ('F21', None, 1.5999999999999988, 0.1)
2021-11-01 23:21:27,510 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
2021-11-01 23:21:27,511 INFO sqlalchemy.engine.Engine [cached since 2.003s ago] ((362, 99.415, 99.415, 99.415, 152883.701, 122585.068), (363, 99.015, 99.015, 99.015, 152883.701, 122585.068), (364, 98.015, 98.015, 98.015, 152883.701, 122585.068), (365, 97.61500000000001, 97.61500000000001, 97.61500000000001, 152883.701, 122585.068))
2021-11-01 23:21:27,512 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-11-01 23:21:27,513 INFO sqlalchemy.engine.Engine [cached since 2.002s ago] ((181, 'F21', 0, 'sample', "{'benzène': 'vi', 'toluène': 'vs', 'plomb': 'vs', 'arsenic': 'vs', 'cuivre': 'vs', 'nickel': 'vs'}", 362, 363, None

2021-11-01 23:21:27,588 INFO sqlalchemy.engine.Engine [cached since 2.059s ago] (370,)
2021-11-01 23:21:27,590 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:27,591 INFO sqlalchemy.engine.Engine [cached since 2.059s ago] (371,)
No lithology interval found, a default one is created for visualization !

Borehole geometry created successfully !

2021-11-01 23:21:27,611 INFO sqlalchemy.engine.Engine INSERT INTO "Boreholes" (id, date, length, diameter) VALUES (?, ?, ?, ?)
2021-11-01 23:21:27,613 INFO sqlalchemy.engine.Engine [cached since 2.109s ago] ('F23', None, 1.7999999999999998, 0.1)
2021-11-01 23:21:27,614 INFO sqlalchemy.engine.Engine INSERT INTO "Positions" (id, upper, middle, lower, x, y) VALUES (?, ?, ?, ?, ?, ?)
20

2021-11-01 23:21:27,696 INFO sqlalchemy.engine.Engine INSERT INTO "Intervals" (id, borehole, interval_number, type, description, top_id, base_id, data_id) VALUES (?, ?, ?, ?, ?, ?, ?, ?)
2021-11-01 23:21:27,697 INFO sqlalchemy.engine.Engine [cached since 2.186s ago] ((189, 'F25', 0, 'sample', "{'benzène': 'vi', 'toluène': 'vs', 'plomb': 'vs', 'arsenic': 'vs', 'cuivre': 'vs', 'nickel': 'vs'}", 378, 379, None), (190, 'F25', 1, 'sample', "{'benzène': 'vi', 'toluène': 'vr', 'plomb': 'vi', 'arsenic': 'vr', 'cuivre': 'vs', 'nickel': 'vs'}", 380, 381, None), (191, 'F25', 2, 'sample', "{'benzène': 'vr', 'toluène': 'vr', 'plomb': 'vr', 'arsenic': 'vr', 'cuivre': 'vr', 'nickel': 'vr'}", 382, 383, None))
2021-11-01 23:21:27,698 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  42
2021-11-01 23:21:27,702 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:27,703 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "

2021-11-01 23:21:27,766 INFO sqlalchemy.engine.Engine [cached since 2.237s ago] (386,)
2021-11-01 23:21:27,768 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:27,769 INFO sqlalchemy.engine.Engine [cached since 2.237s ago] (387,)
2021-11-01 23:21:27,770 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:27,771 INFO sqlalchemy.engine.Engine [cached since 2.242s ago] (388,)
2021-11-01 23:21:27,772 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_i

2021-11-01 23:21:27,863 INFO sqlalchemy.engine.Engine [cached since 2.331s ago] (395,)
2021-11-01 23:21:27,865 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:27,866 INFO sqlalchemy.engine.Engine [cached since 2.337s ago] (396,)
2021-11-01 23:21:27,867 INFO sqlalchemy.engine.Engine SELECT "Positions".id AS "Positions_id", "Positions".upper AS "Positions_upper", "Positions".middle AS "Positions_middle", "Positions".lower AS "Positions_lower", "Positions".x AS "Positions_x", "Positions".y AS "Positions_y" 
FROM "Positions" 
WHERE "Positions".id = ?
2021-11-01 23:21:27,868 INFO sqlalchemy.engine.Engine [cached since 2.336s ago] (397,)
No lithology interval found, a default one is created for visualization !

Borehole geometr

In [31]:
p.add_link_components_intervals(link_intv_comp)

2021-11-01 23:21:28,048 INFO sqlalchemy.engine.Engine INSERT INTO "Linkintervalcomponent" (intv_id, comp_id, extra_data) VALUES (?, ?, ?)
2021-11-01 23:21:28,050 INFO sqlalchemy.engine.Engine [generated in 0.00451s] ((0, 0, ''), (0, 1, ''), (0, 2, ''), (0, 3, ''), (0, 4, ''), (0, 5, ''), (0, 6, ''), (1, 0, '')  ... displaying 10 of 1282 total bound parameter sets ...  (198, 17, ''), (198, 19, ''))
2021-11-01 23:21:28,068 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  46
2021-11-01 23:21:28,074 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:28,075 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-11-01 23:21:28,076 INFO sqlalchemy.engine.Engine [cached since 2.599s ago] ()


#### Validate modifications 

In [32]:
p.commit()

2021-11-01 23:21:28,082 INFO sqlalchemy.engine.Engine COMMIT
Boreholes in the project :  46


In [33]:
p.refresh()

2021-11-01 23:21:28,090 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-11-01 23:21:28,092 INFO sqlalchemy.engine.Engine SELECT "Boreholes".id AS "Boreholes_id", "Boreholes".date AS "Boreholes_date", "Boreholes".length AS "Boreholes_length", "Boreholes".diameter AS "Boreholes_diameter" 
FROM "Boreholes"
2021-11-01 23:21:28,093 INFO sqlalchemy.engine.Engine [cached since 2.617s ago] ()


In [34]:
session.close()

2021-11-01 23:21:28,100 INFO sqlalchemy.engine.Engine ROLLBACK


## update project legend dict

In [35]:
p.update_legend_cmap(compute_all_attrib=True, verbose=False)

In [36]:
p.legend_dict['lithology']['legend']

Legend(Decor({'_colour': '#ffffff', 'width': 3, 'hatch': None, 'component': Component({'lithology': 'not_exist'})}))

## Visualization

In [37]:
n = 1 # borehole index in the project's boreholes list
# df.query(f'ID=="{p.boreholes_3d[n].name}"')[['ID', 'Top_intv', 'Base_intv', 'Lithology']]
df.query(f'ID=="{p.boreholes_3d[n].name}"')

,ID,X,Y,Z,Long_for,Top_intv,Base_intv,Epais_intv,ID_ech,Litho_top,Litho_base,Lithology,Type_ech,Bnz,Toln,Pb,As,Cu,Ni
2,205,152887.693175,122594.619977,101.804167,4.8,0.2,0.6,0.4,205/0,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,0.69,0.47,94.0,18.0,37.0,280.0
3,205,152887.693175,122594.619977,101.804167,4.8,1.9,2.4,0.5,205/1,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,3.10,1.50,590.0,13.0,19.0,30.0
4,205,152887.693175,122594.619977,101.804167,4.8,3.0,3.2,0.2,205/3,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,3.00,0.61,46.0,7.0,13.0,18.0
5,205,152887.693175,122594.619977,101.804167,4.8,3.2,3.6,0.4,205/3b,0.0,0.2,Couche de boue en surface liée à l'interventio...,Sol,0.80,0.13,12.0,10.0,9.6,22.0


In [38]:
attributes = list(legend_dict.keys())
attributes

['lithology', 'benzène', 'toluène', 'plomb', 'arsenic', 'cuivre', 'nickel']

### Log plot for one borehole

In [39]:
at = 2
n = 5
repr_att = attributes[at]

for i in p.boreholes_3d[n].intervals:
    for c in i.components:
        if hasattr(c, repr_att): 
            print(p.boreholes_3d[n].name, '||', i.top.middle, i.base.middle, c)

214 || 0.0 0.1 {'toluène': 'Inconnu'}


In [40]:
p.boreholes_3d[n].intervals

[Interval({'top': Position({'middle': 0.0, 'upper': 0.0, 'lower': 0.0, 'x': 152888.08180400002, 'y': 122588.485566, 'units': 'm'}), 'base': Position({'middle': 0.1, 'x': 152888.08180400002, 'y': 122588.485566, 'units': 'm'}), 'description': '', 'data': {}, 'components': [Component({'lithology': 'not_exist'}), Component({'benzène': 'Inconnu'}), Component({'toluène': 'Inconnu'}), Component({'plomb': 'Inconnu'}), Component({'arsenic': 'Inconnu'}), Component({'cuivre': 'Inconnu'}), Component({'nickel': 'Inconnu'})]})]

In [43]:
p.boreholes_3d[n].components #components #_intervals_dict

{0: [Component({'lithology': 'not_exist'}),
  Component({'benzène': 'Inconnu'}),
  Component({'toluène': 'Inconnu'}),
  Component({'plomb': 'Inconnu'}),
  Component({'arsenic': 'Inconnu'}),
  Component({'cuivre': 'Inconnu'}),
  Component({'nickel': 'Inconnu'})]}

In [ ]:
p.boreholes_3d[n].components

In [ ]:
p.boreholes_3d[n]._intervals_dict

In [ ]:
p.boreholes_3d[n].plot_log(repr_attribute=repr_att)

### 3D display of one borehole

In [ ]:
p.boreholes_3d[n].plot_3d(repr_attribute=repr_att)

### Localization of project boreholes

### 3D display of project boreholes

In [ ]:
pause

In [ ]:
p.plot_3d(repr_attribute=repr_att, labels_size=15, bg_color=["royalblue", "aliceblue"], 
         window_size=(800, 600), verbose=True)